In [4]:
%run "./Includes/Classroom-Setup"

Set up relevant paths.

In [7]:
deltaDataPath = workingDir + "/customer-data-delta/"

## SMALL FILE PROBLEM

Historical and new data is often written in very small files and directories. 

This data may be spread across a data center or even across the world (that is, not co-located).

The result is that a query on this data may be very slow due to
* network latency 
* volume of file metatadata 

The solution is to compact many small files into one larger file.
Databricks Delta has a mechanism for compacting small files.

Use Amazon S3's file browser to see many small files.

S3's file browser is available ONLY in the AWS console (not in Databricks)

### OPTIMIZE
Databricks Delta supports the `OPTIMIZE` operation, which performs file compaction.

Small files are compacted together into new larger files up to 1GB.

`OPTIMIZE` does not do any kind of file clean up, so, at this point the number of files increases!

The 1GB size was determined by the Databricks optimization team as a trade-off between query speed and run-time performance when running Optimize.

`OPTIMIZE` is not run automatically because you must collect many small files first.

* Run `OPTIMIZE` more often if you want better end-user query performance 
* Since `OPTIMIZE` is a time consuming step, run it less often if you want to optimize cost of compute hours
* To start with, run `OPTIMIZE` on a daily basis (preferably at night when spot prices are low), and determine the right frequency for your particular business case
* In the end, the frequency at which you run `OPTIMIZE` is a business decision

The easiest way to see what `OPTIMIZE` does is to perform a simple `count(*)` query before and after and compare the timing!

### Repopulate Data Set

You may have deleted the files created in previous lessons.

We re-create them for you.

In [12]:
from pyspark.sql.functions import expr, col, from_unixtime, to_date
jsonSchema = "action string, time long"
streamingEventPath = "/mnt/training/structured-streaming/events/"
deltaIotPath  = workingDir + "/iot-pipeline/"

rawDataDF = (spark
  .read 
  .schema(jsonSchema)
  .json(streamingEventPath) 
  .withColumn("date", to_date(from_unixtime(col("time").cast("Long"),"yyyy-MM-dd")))
  .withColumn("deviceId", expr("cast(rand(5) * 100 as int)"))
  .repartition(200)
  .write
  .mode("overwrite")
  .format("delta")
  .partitionBy("date")
  .save(deltaIotPath)
)

Take a look at a subdirectory of `deltaIotPath`.

Notice, hundreds of files like `../delta/iot-pipeline/date=xxxx-xx-xx/part-xxxx.snappy.parquet`.

In [14]:
try:
  print(dbutils.fs.ls(dbutils.fs.ls(deltaIotPath)[1].path))
except:
  print("There are no files in deltaIotPath")

[FileInfo(path='dbfs:/user/jose.manuel.bustos.munoz@everis.com/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00000-cbc08ebe-acd5-4cb7-9fb8-e5ba6b623cc1.c000.snappy.parquet', name='part-00000-cbc08ebe-acd5-4cb7-9fb8-e5ba6b623cc1.c000.snappy.parquet', size=2474), FileInfo(path='dbfs:/user/jose.manuel.bustos.munoz@everis.com/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00001-93cf0d75-5a91-4af7-8b0d-7a97cd54d376.c000.snappy.parquet', name='part-00001-93cf0d75-5a91-4af7-8b0d-7a97cd54d376.c000.snappy.parquet', size=2446), FileInfo(path='dbfs:/user/jose.manuel.bustos.munoz@everis.com/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00002-df0bfb20-db5f-4347-ac12-8f9edc8ad3d9.c000.snappy.parquet', name='part-00002-df0bfb20-db5f-4347-ac12-8f9edc8ad3d9.c000.snappy.parquet', size=2494), FileInfo(path='dbfs:/user/jose.manuel.bustos.munoz@everis.com/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00003-9bf57ff3-781a-421b-aaae-caff806fa4e5.c000.snappy.parquet', name='part-00003-9bf57ff3-781a-421b-aaae-caff806fa4e5.c000.snappy.parquet', size=2429), FileInfo(path='dbfs:/user/jose.manuel.bustos.munoz@everis.com/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00004-1e329c73-b52a-4c15-a2a8-4d25e0cfa3ed.c000.snappy.parquet', name='part-00004-1e329c73-b52a-4c15-a2a8-4d25e0cfa3ed.c000.snappy.parquet', size=2425), FileInfo(path='dbfs:/user/jose.manuel.bustos.munoz@everis.com/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00005-7f06ccf0-f17a-4b7a-be51-2f440884ce8d.c000.snappy.parquet', name='part-00005-7f06ccf0-f17a-4b7a-be51-2f440884ce8d.c000.snappy.parquet', size=2470), FileInfo(path='dbfs:/user/jose.manuel.bustos.munoz@everis.com/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00006-6741aaf6-4a5c-4a43-98d9-a1d51ad58c72.c000.snappy.parquet', name='part-00006-6741aaf6-4a5c-4a43-98d9-a1d51ad58c72.c000.snappy.parquet', size=2472), FileInfo(path='dbfs:/user/jose.manuel.bustos.munoz@everis.com/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00007-5e478003-79fe-4b99-a93e-1d76044f9709.c000.snappy.parquet', name='part-00007-5e478003-79fe-4b99-a93e-1d76044f9709.c000.snappy.parquet', size=2546), FileInfo(path='dbfs:/user/jose.manuel.bustos.munoz@everis.com/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00008-73198de7-084f-4248-85f6-5ef1da2f5120.c000.snappy.parquet', name='part-00008-73198de7-084f-4248-85f6-5ef1da2f5120.c000.snappy.parquet', size=2465), FileInfo(path='dbfs:/user/jose.manuel.bustos.munoz@everis.com/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00009-4d4dcf9c-ab7f-4138-ab25-3d9d9316944e.c000.snappy.parquet', name='part-00009-4d4dcf9c-ab7f-4138-ab25-3d9d9316944e.c000.snappy.parquet', size=2468), FileInfo(path='dbfs:/user/jose.manuel.bustos.munoz@everis.com/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00010-0982796e-36d3-41cc-9954-9d07da5af5e3.c000.snappy.parquet', name='part-00010-0982796e-36d3-41cc-9954-9d07da5af5e3.c000.snappy.parquet', size=2500), FileInfo(path='dbfs:/user/jose.manuel.bustos.munoz@everis.com/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00011-c9f8bf73-1eec-4431-86d9-1ede804c7110.c000.snappy.parquet', name='part-00011-c9f8bf73-1eec-4431-86d9-1ede804c7110.c000.snappy.parquet', size=2494), FileInfo(path='dbfs:/user/jose.manuel.bustos.munoz@everis.com/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00012-bd5225d1-7772-4718-89ab-849fc14c9e46.c000.snappy.parquet', name='part-00012-bd5225d1-7772-4718-89ab-849fc14c9e46.c000.snappy.parquet', size=2543), FileInfo(path='dbfs:/user/jose.manuel.bustos.munoz@everis.com/delta/delta_06_optimization_psp/iot-pipeline/date=2016-07-26/part-00013-5b8337b8-b396-4e9e-b69d-31b9defb02eb.c000.snappy.parquet', name='part-00013-5b8337b8-b396-4e9e-b69d-31b9defb02eb.c000.snappy.parquet', size=2492), FileInfo(path='dbfs:/user/jose.manuel.bustos.munoz@everis.com/del

Pick a `deviceId` then run the `SELECT` query. 

Notice it is very slow, due to the large number of small files.

In [16]:
devID = spark.sql("SELECT deviceId FROM delta.`{}` limit 1".format(deltaIotPath)).first()[0]
iotDF = spark.sql("SELECT * FROM delta.`{}` where deviceId={}".format(deltaIotPath, devID))
display(iotDF)

action,time,date,deviceId
Open,1469612542,2016-07-27,19
Close,1469587626,2016-07-27,19
Open,1469591219,2016-07-27,19
Close,1469644952,2016-07-27,19
Close,1469588369,2016-07-27,19
Open,1469663037,2016-07-27,19
Close,1469623631,2016-07-27,19
Close,1469654838,2016-07-27,19
Open,1469631369,2016-07-27,19
Close,1469605146,2016-07-27,19


### Partition Pruning, Data Skipping and ZORDER

Databricks Delta uses multiple mechanisms to speed up queries.

<b>Partition Pruning</b> is a performance optimization that speeds up queries by limiting the amount of data read.

If the WHERE clause filters on a table partitioned column, then only table partitions (sub-directories) that may have matching records are read.  

For example, we have a data set that is partitioned by `date`. 

A query using `WHERE date > 2018-06-01` would not access data that resides in partitions that correspond to dates prior to `2018-06-01`.  

<b>Data Skipping</b> is a performance optimization that aims at speeding up queries that contain filters (WHERE clauses). 

As new data is inserted into a Databricks Delta table, file-level min/max statistics are collected for all columns (including nested ones) of supported types. Then, when there’s a lookup query against the table, Databricks Delta first consults these statistics in order to determine which files can safely be skipped.

<b>ZOrdering</b> is a technique to colocate related information in the same set of files. 

ZOrdering maps multidimensional data to one dimension while preserving locality of the data points. 

Given a column that you want to perform ZORDER on, say `OrderColumn`, Delta
* Takes existing parquet files within a partition.
* Maps the rows within the parquet files according to `OrderColumn` using <a href="https://en.wikipedia.org/wiki/Z-order_curve" target="_blank">this algorithm</a>.
* In the case of only one column, the mapping above becomes a linear sort.
* Rewrites the sorted data into new parquet files.

You cannot use the table partition column also as a ZORDER column.

### ZORDER example
In the image below, table `Students` has 4 columns: 
* `gender` with 2 distinct values
* `Pass-Fail` with 2 distinct values
* `Class` with 4 distinct values  
* `Student` with many distinct values 

Suppose you wish to perform the following query:

```SELECT Name FROM Students WHERE gender = 'M' AND Pass_Fail = 'P' AND Class = 'Junior'```

```ORDER BY Gender, Pass_Fail```

The most effective way of performing that search is to order the data starting with the largest set, which is `Gender` in this case. 

If you're searching for `gender = 'M'`, then you don't even have to look at students with `gender = 'F'`. 

Note that this technique is only beneficial if all `gender = 'M'` values are co-located.

### ZORDER usage

With Databricks Delta the notation is:

> `OPTIMIZE Students`<br>
`ZORDER BY Gender, Pass_Fail`

This will ensure all the data backing `Gender = 'M' ` is colocated, then data associated with `Pass_Fail = 'P' ` is colocated.

See References below for more details on the algorithms behind ZORDER.

Using ZORDER, you can order by multiple columns as a comma separated list; however, the effectiveness of locality drops.

In streaming, where incoming events are inherently ordered (more or less) by event time, use `ZORDER` to sort by a different column, say 'userID'.

In [20]:
spark.sql("""OPTIMIZE delta.`{}` 
             ZORDER by (deviceID)""".format(deltaIotPath))

Out[17]: DataFrame[path: string, metrics: struct<filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,mergedNumCubes:bigint>,numBatches:bigint>]

The performance of the following query should now be much faster than it was before.

In [22]:
deviceDF = spark.sql("SELECT * FROM delta.`{}` WHERE deviceId={}".format(deltaIotPath, devID))
display(deviceDF)

action,time,date,deviceId
Open,1469612542,2016-07-27,19
Close,1469587626,2016-07-27,19
Open,1469591219,2016-07-27,19
Close,1469644952,2016-07-27,19
Close,1469588369,2016-07-27,19
Open,1469663037,2016-07-27,19
Close,1469623631,2016-07-27,19
Close,1469654838,2016-07-27,19
Open,1469631369,2016-07-27,19
Close,1469605146,2016-07-27,19


## VACUUM

To save on storage costs you should occasionally clean up invalid files using the `VACUUM` command. 

Invalid files are small files compacted into a larger file with the `OPTIMIZE` command.

The  syntax of the `VACUUM` command is 
>`VACUUM name-of-table RETAIN number-of HOURS;`

The `number-of` parameter is the <b>retention interval</b>, specified in hours.

Databricks does not recommend you set a retention interval shorter than seven days because old snapshots and uncommitted files can still be in use by concurrent readers or writers to the table.

The scenario here is:
0. User A starts a query off uncompacted files, then
0. User B invokes a `VACUUM` command, which deletes the uncompacted files
0. User A's query fails because the underlying files have disappeared

Invalid files can also result from updates/upserts/deletions.

Count the number of files before we vacuum.

In [24]:
try:
  print(len(dbutils.fs.ls(dbutils.fs.ls(deltaIotPath)[1].path)))
except:
  print("There are no files in deltaIotPath")

201

In the example below we set off an immediate `VACUUM` operation with an override of the retention check so that all files are cleaned up immediately.

You would not do not do this in production!

If you do not set the override of the retention check, you get a helpful message.

```
requirement failed: Are you sure you would like to vacuum files with such a low retention period? If you have
writers that are currently writing to this table, there is a risk that you may corrupt the
state of your Delta table.

If you are certain that there are no operations being performed on this table, such as
insert/upsert/delete/optimize, then you may turn off this check by setting:
spark.databricks.delta.retentionDurationCheck.enabled = False
```

In [26]:
try:
  spark.sql(""" VACUUM delta.`{}` RETAIN 0 HOURS """.format(deltaIotPath))
except Exception as err: 
  print(str(err).replace("\\n", "\n").replace("'", ""))

requirement failed: Are you sure you would like to vacuum files with such a low retention period? If you have
writers that are currently writing to this table, there is a risk that you may corrupt the
state of your Delta table.

If you are certain that there are no operations being performed on this table, such as
insert/upsert/delete/optimize, then you may turn off this check by setting:
spark.databricks.delta.retentionDurationCheck.enabled = false

If you are not sure, please use a value not less than "168 hours".

In [27]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", False)

spark.sql(" VACUUM delta.`{}` RETAIN 0 HOURS ".format(deltaIotPath))

Out[21]: DataFrame[path: string]

Notice how the directory looks vastly cleaned up!

In [29]:
try:
  print(len(dbutils.fs.ls(dbutils.fs.ls(deltaIotPath)[1].path)))
except:
  print("There are no files in deltaIotPath")

1

# LAB

## Step 1: Repopulate Data

If you've deleted the data under `deltaDataPath = workingDir + "/customer-data-delta/"` in previous lessons, that is okay.

If the path no longer exists or has been cleaned out, repopulate data, otherwise, do nothing.

We are chaining the `read` and `write` operations in one statement.

In [32]:
inputPath = "/mnt/training/online_retail/data-001/data.csv"
inputSchema = "InvoiceNo STRING, StockCode STRING, Description STRING, Quantity INT, InvoiceDate STRING, UnitPrice DOUBLE, CustomerID INT, Country STRING"
deltaDataPath = workingDir + "/customer-data-delta/"

(spark.read
  .option("header", "true")
  .schema(inputSchema)
  .csv(inputPath) 
  .write
  .mode("overwrite")
  .format("delta")
  .partitionBy("Country")
  .save(deltaDataPath)  )

## Step 2: Time an Unoptimized Query by StockCode

Let's apply some of these optimizations to the `customer_data_delta` table.

Our data is partitioned by `Country`.

We want to query the data for `StockCode` equal to `22301`. 

We expect this query to be slow because we have to examine ALL OF the underlying data in `customer_data_delta` to find the desired `StockCode`. 

The data could be found on servers all over the world! That is, data can be coming from replications in other zones.

First, let's time the above query: you will need to form a DataFrame to pass to `preZorderQuery`.

 In Python we use the `timeit` 
<a href="https://docs.python.org/2/library/timeit.html" target="_blank">library function</a> 



Note that `timeit`  takes a function as input, so you need to define `myQuery` as a function.

In [34]:
# TODO
import timeit
deltaDataPath = workingDir + "/customer-data-delta/"

def myQuery():
  return """spark.sql("SELECT * FROM delta.`{}` WHERE StockCode=22301 ".format(deltaDataPath)).collect()"""

preTime = timeit.timeit(myQuery)

## Step 3: OPTIMIZE and ZORDER

Let's apply some of Databricks Delta's optimizations to `customer_data_delta`.

Our data is partitioned by `Country`.

Compact the files and re-order by `StockCode`.

In [36]:
spark.sql("""OPTIMIZE delta.`{}` 
             ZORDER by (stockCode)""".format(deltaDataPath))

Out[25]: DataFrame[path: string, metrics: struct<filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,mergedNumCubes:bigint>,numBatches:bigint>]

## Step 4: Time an Optimized Query by StockCode

Let's time the above query again: use the same methodology as for the pre-optimized query.

We expect `postTime` to be smaller than `preTime`.

Recall, you defined `myQuery` previously.

In [38]:
import timeit

postTime = timeit.timeit(myQuery)

In [39]:
# TEST  - Run this cell to test your solution.
print("Pre ZORDER time is {} s".format(preTime))
print("Post ZORDER time is {} s".format(postTime))

Pre ZORDER time is 0.08212589399772696 s
Post ZORDER time is 0.08126123301917687 s

## Step 5: Apply VACUUM

Make sure you set the retention period to 0 to perform the operation immediately.

There should be only 1 file in each `Country` partition.

In [41]:
spark.sql("VACUUM delta.`{}` RETAIN 0 HOURS".format(deltaDataPath))

Out[28]: DataFrame[path: string]

In [42]:
# TEST - Run this cell to test your solution.
from functools import reduce
try:
  myList = filter(lambda p: "_delta_log" not in p.path, dbutils.fs.ls(deltaDataPath))  # Pick up list of subdirectories except for _delta_log
  myMap = map(lambda p: computeFileStats(p.path)[0], myList)                           # computeFileStats is a tuple of (numFiles, fileSize)
  numFilesOne = reduce(lambda a, b: a and b and 1, myMap)                              # AND every element with eachother and the value 1
except:
  numFilesOne = -99

dbTest("Delta-08-numFilesOne", 1, numFilesOne)

print("Tests passed!")

Tests passed!

In [44]:
%run "./Includes/Classroom-Cleanup"

## Review Questions

**Q:** Why are many small files problematic when doing queries on data backed by these?<br>
**A:** If there are many files, some of which might not be co-located, the principal sources of slowdown are
* network latency 
* (volume of) file metatadata 

**Q:** What do `OPTIMIZE` and `VACUUM` do?<br>
**A:** `OPTIMIZE` create larger files from a collection of smaller files and `VACUUM` deletes the invalid small files that were used in compaction.

**Q:** What size files does `OPTIMIZE` compact to and why that value?<br>
**A:** Small files are compacted to around 1GB; this value was determined by the Spark optimization team as a good compromise between speed and performace.

**Q:** What should one be careful of when using `VACUUM`?<br>
**A:** Don't set a retention interval shorter than seven days because old snapshots and uncommitted files can still be in use by concurrent readers or writers to the table.

**Q:** What does `ZORDER` do?<br>
**A:** It is a technique to colocate related information in the same set of files to improve query performance.